In [437]:
VID = 0x0403
PID = 0x6030
I2C_INTERFACE = 0

In [6]:
import libusb as usb

usbd = usb.init(None)
dev = usb.open_device_with_vid_pid(None, VID, PID)
if usb.kernel_driver_active(dev, 0):
    print("Kernel has device, detaching...")
    usb.detach_kernel_driver(dev, 0)
usb.close(dev)


Kernel has device, detaching...


In [438]:
import hid
import time

device = hid.device()
device.open(VID, PID)

In [204]:
devices = hid.enumerate()

for d in devices:
    print(d)
    if d['vendor_id'] == VID and d['product_id'] == PID:
        print("Nalezeno zařízení:", d)

        if d['interface_number'] == I2C_INTERFACE:
            try:
                print(d['path'])
                device = hid.device()
                device.open_path(d['path'])
            finally:
                print("Vybrane zarizeni", device)
                #device.close()

{'path': b'3-8:1.0', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 0}
Nalezeno zařízení: {'path': b'3-8:1.0', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 0}
b'3-8:1.0'
Vybrane zarizeni <hid.device object at 0x7fcc87960880>
{'path': b'3-8:1.1', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 1}
Nalezeno zařízení: {'path': b'3-8:1.1', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 1}
{'path': b'5-2.3.5:1.0', 'vendor

In [201]:
device.close()

In [481]:

class FT260HidDriver():

    """
    Key to symbols
    ==============

    S     (1 bit) : Start bit
    P     (1 bit) : Stop bit
    Rd/Wr (1 bit) : Read/Write bit. Rd equals 1, Wr equals 0.
    A, NA (1 bit) : Accept and reverse accept bit.
    Addr  (7 bits): I2C 7 bit address. Note that this can be expanded as usual to
                    get a 10 bit I2C address.
    Comm  (8 bits): Command byte, a data byte which often selects a register on
                    the device.
    Data  (8 bits): A plain data byte. Sometimes, I write DataLow, DataHigh
                    for 16 bit data.
    Count (8 bits): A data byte containing the length of a block operation.

    [..]: Data sent by I2C device, as opposed to data sent by the host adapter.

    More detail documentation is at https://www.kernel.org/doc/Documentation/i2c/smbus-protocol
    """

    def __init__(self, port, device):
        self.port = port
        #self.smbus = smbus
        self.driver_type = 'ft260_hid'
        self.device = device
        self.initialize_ftdi()
    


    def initialize_ftdi(self):
        # TODO pripojeni k HID, nyni to mam jako self.device
        
        print(f'Device manufacturer: {self.device.get_manufacturer_string()}')
        print(f'Product: {self.device.get_product_string()}')
        print(f'Serial Number: {self.device.get_serial_number_string()}')

        self.device.set_nonblocking(0)

        self.reset_i2c()
        #self.set_i2c_speed(100000) # 100 Khz
        self.get_i2c_status()


    def get_i2c_status(self):
        d = self.device.get_feature_report(0xC0, 100)

        status = ['busy_chip', 'error', 'no_ack', 'arbitration_lost', 'idle', 'busy_bus']
        bits = [(d[1] & (1 << i)) >> i for i in range(8)]
        status = dict(zip(status, bits))

        baudrate = (d[2] | d[3]<<8)*1000
        status['baudrate'] = baudrate

        return status
        
    
    def reset_i2c(self):
        self.device.send_feature_report([0xA1, 0x20])
        
    def set_i2c_speed(self, speed = 100000):
        speed = int(speed/1000)
        LSB = (speed & 0xff)
        MSB = (speed>>8 & 0xff)
        print(f"Nastavit speed na {speed} Hz: ", hex(LSB), hex(MSB))
        self.device.send_feature_report([0xA1, 0x22, LSB, MSB])


    def write_byte(self, address, value):
        """
        SMBus Send Byte:  i2c_smbus_write_byte()
        ========================================

        This operation is the reverse of Receive Byte: it sends a single byte
        to a device.  See Receive Byte for more information.

        S Addr Wr [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_BYTE
        """

        return None

    def read_byte(self, address):
        """
        SMBus Send Byte:  i2c_smbus_write_byte()
        ========================================

        This operation is the reverse of Receive Byte: it sends a single byte
        to a device.  See Receive Byte for more information.

        S Addr Wr [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_BYTE
        """
        return None

    def write_byte_data(self, address, register, value):
        """
        SMBus Read Byte:  i2c_smbus_read_byte_data()
        ============================================

        This reads a single byte from a device, from a designated register.
        The register is specified through the Comm byte.

        S Addr Wr [A] Comm [A] S Addr Rd [A] [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_BYTE_DATA
        """

        return None

    def read_byte_data(self, address, register):
        """
        SMBus Read Byte:  i2c_smbus_read_byte_data()
        ============================================

        This reads a single byte from a device, from a designated register.
        The register is specified through the Comm byte.

        S Addr Wr [A] Comm [A] S Addr Rd [A] [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_BYTE_DATA
        """


        payload = [0xD0, address, 0x06, 0b01, register]
        self.device.write(payload)
        length = (1).to_bytes(2, byteorder='little')
        self.device.write([0xC2, address, 0x06, length[0], length[1]])
        d = self.device.read(0xde)

        # TODO: Osetrit chyby v chybnem vycteni registru
        return d[2]

    def write_word_data(self, address, register, value):
        """
        SMBus Write Word:  i2c_smbus_write_word_data()
        ==============================================

        This is the opposite of the Read Word operation. 16 bits
        of data is written to a device, to the designated register that is
        specified through the Comm byte.

        S Addr Wr [A] Comm [A] DataLow [A] DataHigh [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_WORD_DATA

        Note the convenience function i2c_smbus_write_word_swapped is
        available for writes where the two data bytes are the other way
        around (not SMBus compliant, but very popular.)
        """
        return None

    def read_word_data(self, address, register):
        """
        SMBus Read Word:  i2c_smbus_read_word_data()
        ============================================

        This operation is very like Read Byte; again, data is read from a
        device, from a designated register that is specified through the Comm
        byte. But this time, the data is a complete word (16 bits).

        S Addr Wr [A] Comm [A] S Addr Rd [A] [DataLow] A [DataHigh] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_WORD_DATA

        Note the convenience function i2c_smbus_read_word_swapped is
        available for reads where the two data bytes are the other way
        around (not SMBus compliant, but very popular.)
        """

        payload = [0xD0, address, 0x06, 0b01, register]
        self.device.write(payload)
        length = (2).to_bytes(2, byteorder='little')
        self.device.write([0xC2, address, 0x06, length[0], length[1]])
        d = self.device.read(0xde)

        # TODO: Osetrit chyby v chybnem vycteni registru
        return d[2]<<8 | d[3]

    def write_block_data(self, address, register, value):
        """
        SMBus Block Write:  i2c_smbus_write_block_data()
        ================================================

        The opposite of the Block Read command, this writes up to 32 bytes to
        a device, to a designated register that is specified through the
        Comm byte. The amount of data is specified in the Count byte.

        S Addr Wr [A] Comm [A] Count [A] Data [A] Data [A] ... [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_BLOCK_DATA
        """
        return None

    def read_block_data(self, address, register):
        """
        SMBus Block Read:  i2c_smbus_read_block_data()
        ==============================================

        This command reads a block of up to 32 bytes from a device, from a
        designated register that is specified through the Comm byte. The amount
        of data is specified by the device in the Count byte.

        S Addr Wr [A] Comm [A]
                   S Addr Rd [A] [Count] A [Data] A [Data] A ... A [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_BLOCK_DATA
        """
        return None

    def block_process_call(self, address, register, value):
        """
        SMBus Block Write - Block Read Process Call
        ===========================================

        SMBus Block Write - Block Read Process Call was introduced in
        Revision 2.0 of the specification.

        This command selects a device register (through the Comm byte), sends
        1 to 31 bytes of data to it, and reads 1 to 31 bytes of data in return.

        S Addr Wr [A] Comm [A] Count [A] Data [A] ...
                                     S Addr Rd [A] [Count] A [Data] ... A P

        Functionality flag: I2C_FUNC_SMBUS_BLOCK_PROC_CALL
        """
        return None

    ### I2C transactions not compatible with pure SMBus driver
    def write_i2c_block(self, address, value):
        """
        Simple send transaction
        ======================

        This corresponds to i2c_master_send.

          S Addr Wr [A] Data [A] Data [A] ... [A] Data [A] P

        More detail documentation is at: https://www.kernel.org/doc/Documentation/i2c/i2c-protocol
        """
        return None

    def read_i2c_block(self, address, length):
        """
        Simple receive transaction
        ===========================

        This corresponds to i2c_master_recv

          S Addr Rd [A] [Data] A [Data] A ... A [Data] NA P

        More detail documentation is at: https://www.kernel.org/doc/Documentation/i2c/i2c-protocol
        """
        return None

    def write_i2c_block_data(self, address, register, value):
        """
        I2C block transactions do not limit the number of bytes transferred
        but the SMBus layer places a limit of 32 bytes.

        I2C Block Write:  i2c_smbus_write_i2c_block_data()
        ==================================================

        The opposite of the Block Read command, this writes bytes to
        a device, to a designated register that is specified through the
        Comm byte. Note that command lengths of 0, 2, or more bytes are
        seupported as they are indistinguishable from data.

        S Addr Wr [A] Comm [A] Data [A] Data [A] ... [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_I2C_BLOCK
        """
        return None

    def read_i2c_block_data(self, address, register, length):
        """
        I2C block transactions do not limit the number of bytes transferred
        but the SMBus layer places a limit of 32 bytes.

        I2C Block Read:  i2c_smbus_read_i2c_block_data()
        ================================================

        This command reads a block of bytes from a device, from a
        designated register that is specified through the Comm byte.

        S Addr Wr [A] Comm [A]
                   S Addr Rd [A] [Data] A [Data] A ... A [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_I2C_BLOCK
        """
        return None


c = FT260HidDriver(18, device)

def set_time(time):
    c.write_byte_data(addr_rtc, 0x01, (time.second % 10) | int(time.second/10)<<4 )
    c.write_byte_data(addr_rtc, 0x02, (time.minute % 10) | int(time.minute/10)<<4 )
    c.write_byte_data(addr_rtc, 0x03, (time.hour % 10) | int(time.hour/10)<<4 )
    c.write_byte_data(addr_rtc, 0x04, (time.day % 10) | int(time.day/10)<<4 )
    c.write_byte_data(addr_rtc, 0x06, (time.month % 10) | int(time.month/10)<<4 )
    c.write_byte_data(addr_rtc, 0x07, ((time.year-2000) % 10) | int((time.year-2000)/10)<<4 )

set_time(datetime.datetime.utcnow())

addr_rtc = 0x51
r01 = c.read_byte_data(addr_rtc, 0x01)
r02 = c.read_byte_data(addr_rtc, 0x02)
r03 = c.read_byte_data(addr_rtc, 0x03)
r04 = c.read_byte_data(addr_rtc, 0x04)
r06 = c.read_byte_data(addr_rtc, 0x06)
r07 = c.read_byte_data(addr_rtc, 0x07)

sec = ((r01 >> 4) & 0b111) * 10 + (r01 & 0b1111)
minu= ((r02 >> 4) & 0b111) * 10 + (r02 & 0b1111)
hour= ((r03 >> 4) & 0b11) * 10 + (r03 & 0b1111)
day = ((r04 >> 4) & 0b11) * 10 + (r04 & 0b1111)
mon = ((r06 >> 4) & 0b1) * 10 + (r06 & 0b1111)
year= ((r07 >> 4) & 0b1111) * 10 + (r07 & 0b1111)+2000

print(sec, minu, hour, day, mon, year)

Device manufacturer: FTDI
Product: FT260
Serial Number: Љ
STATUS ['0xc0', '0x20', '0x64', '0x0', '0x3b']
{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1}
25600
51 23 0 1 1 2000


Device manufacturer: FTDI
Product: FT260
Serial Number: Љ
STATUS ['0xc0', '0x20', '0x64', '0x0', '0x3b']
{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1}
25600


In [470]:
import datetime


 
READ
priajto ['0xd0', '0x1', '0x91', '0x1', '0x0', '0x0', '0x32', '0x0']
 
READ
priajto ['0xd0', '0x1', '0x23', '0x1', '0x0', '0x0', '0x32', '0x0']
 
READ
priajto ['0xd0', '0x1', '0x0', '0x1', '0x0', '0x0', '0x32', '0x0']
 
READ
priajto ['0xd0', '0x1', '0x1', '0x1', '0x0', '0x0', '0x32', '0x0']
 
READ
priajto ['0xd0', '0x1', '0x1', '0x1', '0x0', '0x0', '0x32', '0x0']
 
READ
priajto ['0xd0', '0x1', '0x0', '0x1', '0x0', '0x0', '0x32', '0x0']
145 35 0 1 1 0
11 23 0 1 1 2000


In [90]:
speed = 1000
print(hex(speed & 0xff))
print(hex(speed>>8 & 0xff))

0xe8
0x3


In [446]:
def get_i2c_status():
    d = device.get_feature_report(0xC0, 100)
    #print([hex(x) for x in d])
    
    status = ['busy_chip', 'error', 'no_ack', 'arbitration_lost', 'idle', 'busy_bus']
    bits = [(d[1] & (1 << i)) >> i for i in range(8)]
    status = dict(zip(status, bits))
    

    baudrate = (d[2] | d[3]<<8)*1000
    status['baudrate'] = baudrate

    print(status)
    return status 

In [451]:

get_i2c_status()

print(" ")
address = 0x51
register = 0x01


# address = 0x55
# register = 0x10

payload = [0xD0, address, 0x06, 0x01, register, 0x00, 0x00, 0x00]
print("READ BYTE DATA...", [hex(x) for x in payload])
device.write(payload)

get_i2c_status()
time.sleep(1)

length = 1
length = (length).to_bytes(2, byteorder='little')
time.sleep(0.1)
device.write([0xC2, address, 0x06, length[0], length[1]])
time.sleep(0.1)
d = device.read(0xde)
print("READ")
print(d[1], d[2])


print(f"priajto", [hex(x) for x in d])

{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1, 'baudrate': 100000}
 
READ BYTE DATA... ['0xd0', '0x51', '0x6', '0x1', '0x1', '0x0', '0x0', '0x0']
{'busy_chip': 1, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 0, 'baudrate': 100000}
READ
1 161
priajto ['0xd0', '0x1', '0xa1', '0x1', '0x0', '0x0', '0x0', '0x0']


In [445]:
get_i2c_status()

{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1, 'baudrate': 25600000}


{'busy_chip': 0,
 'error': 0,
 'no_ack': 0,
 'arbitration_lost': 0,
 'idle': 0,
 'busy_bus': 1,
 'baudrate': 25600000}

In [449]:
speed = 100000
speed = int(speed/1000)
LSB = (speed & 0xff)
MSB = (speed>>8 & 0xff)
print(f"Nastavit speed na {speed} kHz: ", hex(LSB), hex(MSB))
print("Payload", [hex(x) for x in [0xA1, 0x22, LSB, MSB]])
device.send_feature_report([0xA1, 0x22, LSB, MSB])

Nastavit speed na 100 kHz:  0x64 0x0
Payload ['0xa1', '0x22', '0x64', '0x0']


4

In [427]:
device.send_feature_report([0xA1, 0x20])
device.send_feature_report([0xA1, 0x02, 0x01])

3